# Một hàng đợi hay là hai?

*Mô hình hóa và mô phỏng bằng Python*

Bản quyền 2021 Allen Downey

Giấy phép: [Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International](https://creativecommons.org/licenses/by-nc-sa/4.0/)

In [1]:
# cài đặt Pint nếu cần

try:
    import pint
except ImportError:
    !pip install pint

In [2]:
# tải về modsim.py nếu cần

from os.path import exists

filename = 'modsim.py'
if not exists(filename):
    from urllib.request import urlretrieve
    url = 'https://raw.githubusercontent.com/AllenDowney/ModSim/main/'
    local, _ = urlretrieve(url+filename, filename)
    print('Downloaded ' + local)

In [3]:
# nhập các hàm từ modsim

from modsim import *

Cuốn sổ tính này trình bày nghiên cứu cụ thể trong *Mô hình hóa và mô phỏng bằng Python*.  Mục đích là nhằm khám phá một câu hỏi liên quan tới lý thuyết hàng đợi, vốn đi tìm hiểu những hệ thống có thời gian chờ đợi trong các hàng, còn gọi là những "hàng đợi".

Giả sử bạn dang thiết kế khu vực thanh toán của một quầy hàng. Có đủ chỗ trong quầy để bạn bố trí 2 bàn thanh toán và một khu dành cho khách hàng đứng đợi. Bạn có thể bố trí khách xếp hai hàng, mỗi hàng tiến vào một bàn thanh toán, hay chỉ một hàng đi vào cả 2 bàn.

Về lý thuyết, bạn sẽ dự trù rằng một hàng hẳn phải tốt hơn, nhưng nó có những trở ngại rất thực tế: để duy trì dược một hàng, bạn phải đặt thêm rào chắn, và khách hàng có thể cảm thấy bực vì hàng có vẻ dài hơn, dù rằng hàng này sẽ di chuyển nhanh hơn.

Như vậy bạn sẽ muốn kiểm tra xem liệu một hàng có thực sự nhanh hơn không và nếu có thì nhanh hơn bao nhiêu. Mô phỏng có thể giúp ta giải đáp điều này.

Như đã làm với mô hình chung xe, ta sẽ giả sử rằng một khách hàng có khả năng đồng đều xuất hiện vào bất kì bước thời gian nào. Tôi sẽ kí hiệu xác suất này bằng chữ cái Hi Lạp lambda, $\lambda$, hoặc bằng tên biến `lam`.  Trị số của $\lambda$ có lẽ thay đổi ngày qua ngày, bởi vậy ta sẽ phải xét một khoảng các xác suất.

Dựa vào số liệu từ những cửa hàng khác, bạn biết rằng trung bình mỗi khách hàng phải mất 5 phút để chờ đến khi thanh toán xong. Nhưng thời gian thanh toán này lại khác nhau: đa số khách hàng mất ít hơn 5 phút, nhưng có người lại mất nhiều hơn 5 phút rất nhiều. Một cách dơn giản để mô hình hóa sự biến đổi này là giả sử rằng khi một khách hàng thanh toán, luôn có cùng xác suất để thanh toán xong xuôi ở bước thời gian kế tiếp, bất kể họ đã mất bao nhiêu thời gian từ lúc xếp hàng đến giờ. Tôi sẽ kí hiệu xác suất này bằng chữ cái Hi Lạp $\mu$, hay là tên biến `mu`.

Nếu ta chọn $\mu=1/5$, mỗi phút, thì thời gian trung bình của mỗi lượt thanh toán sẽ là 5 phút; điều này thống nhất với số liệu ta có.

## Một quầy phục vụ, một hàng đợi

Hãy viết một hàm có tên `make_system`, hàm này nhận vào các tham số `lam` và `mu` rồi trả lại một đối tượng `System` với các biến `lam`, `mu`, và `duration`.  Hãy đặt `duration`, vốn là số bước thời gian mô hỏng, thành 10 giờ (biểu diễn bằng số phút).

In [4]:
# Lời giải

Hãy kiểm tra hàm này bằng cách tạo một đối tượng `System` với `lam=1/8` và `mu=1/5`.

In [5]:
# Lời giải

Hãy viết một hàm cập nhật, hàm này nhận tham số `x`, vốn là tổng số khách trong cửa hàng, tính cả người khách đang thanh toán; `t`, số phút đã trôi qua từ khi bắt đầu mô phỏng, và `system`, vốn là đối tượng `System`.

Nếu có một người khách đang thanh toán, mã lệnh cần dùng `flip` để quyết định xem họ xong chưa. Và cũng dùng `flip` để quyết định xem có người khách nào mới đến không.

Hàm cần trả lại tổng số khách hàng ở cuối bước thời gian.

In [6]:
# Lời giải

Hãy kiểm tra hàm này bằng cách gọi nó với `x=1`, `t=0`, và đối tượng `System` bạn vừa tạo. Nếu chạy nó vài lần, bạn sẽ phải thấy các kết quả khác nhau.

In [7]:
# Lời giải

Bây giờ ta có thể chạy mô phỏng. Dưới đây là một phiên bản `run_simulation` tạo ra một `TimeSeries` với tổng số khách trong cửa hàng, kể cả người đang thanh toán.

In [8]:
def run_simulation(system, update_func):
    """Mô phỏng hệ thống hàng đợi.
    
    system: đối tượng System
    update_func: đối tượng hàm
    """
    x = 0
    results = TimeSeries()
    results[0] = x
    
    for t in linrange(0, system.duration):
        x = update_func(x, t, system)
        results[t+1] = x

    return results

Hãy gọi `run_simulation` với hàm cập nhật bạn vừa viết và vẽ đồ thị kết quả.

In [9]:
# Lời giải

Sau mô phỏng này, ta có thể tính `L`, tức là số khách hàng trung bình trong hệ thống, và `W`, thời gian trung bình người khách ở trong cửa hàng. `L` và `W` được liên hệ bởi định luật Little:

$L = \lambda W$

trong đó $\lambda$ là độ thường xuyên khách đến. Dưới đây là một hàm để tính `L` và `W`.

In [10]:
def compute_metrics(results, system):
    """Tính số khách và thời gian đợi trung bình.
    
    results: một chuỗi thời gian (TimeSeries) của chiều dài hàng đợi
    system: đối tượng System
    
    returns: L, W
    """
    L = results.mean()
    W = L / system.lam
    return L, W

Hãy gọi `compute_metrics` với các kết quả từ mô phỏng của bạn.

In [11]:
# Lời giải

### Quét tham số

Vì không biết giá trị thật của $\lambda$, ta có thể quét qua một khoảng các khả năng, từ 10% đến 80% của tốc độ hoàn thành, $\mu$.  (Nếu khách đến nhanh/thường xuyên hơn là tốc độ hoàn thành thì hàng đợi cứ kéo dài mãi không có giới hạn. Trong trường hợp đó `L` và `W` chỉ tùy thuộc cửa hàng mở cửa bao lâu.)

Hãy tạo một mảng các giá trị cho `lam`.

In [12]:
# Lời giải

Hãy viết một hàm nhận vào một mảng các giá trị của `lam`, một giá trị của `mu`, và một hàm cập nhật.

Với từng giá trị của `lam`, hàm này phải chạy mô phỏng, tính ra `L` và `W`, rồi lưu giá trị của `W` vào một dãy `SweepSeries`.

Hàm cần trả lại `SweepSeries`.

In [13]:
# Lời giải

Hãy gọi hàm bạn vừa viết để tạo một `SweepSeries`, rồi vẽ biểu đồ của nó.

In [14]:
# Lời giải

In [15]:
# Lời giải

Nếu tưởng tượng rằng khoảng giá trị này biểu diễn cho mức thường xuyên vào các ngày khác nhau, ta có thể dùng giá trị trung bình của `W`, cho một dãy các giá trị của `lam`, để so sánh các chiến lược xếp hàng khác nhau.

In [16]:
# Lời giải

### Phân tích

Với hệ thống này tôi đã chọn một mô hình thông dụng trong lý thuyết hàng đợi, phần là vì ta có thể dùng phép toán để suy ra các thuộc tính mô hình.

Cụ thể, ta có thể lập được thời gian trung bình trong quầy như một hàm số theo $\mu$ và $\lambda$:

$W = 1 / (\mu - \lambda)$

Hàm sau đây vẽ đồ thị của $W$ theo $\lambda$.

In [19]:
def plot_W(lam_array, mu):
    """Vẽ đồ thị thời gian đợi trung bình theo lý thuyết
    
    lam_array: mảng giá trị của `lam`
    mu: xác suất thanh toán xong
    """
    W_array = 1 / (mu - lam_array)
    W_series = make_series(lam_array, W_array)
    W_series.plot(style='-', label='analysis')

Hãy dùng hàm này để vẽ biểu đồ kết quả lý thuyết, sau đó trên cùng biểu đồ hãy vẽ chồng lên kết quả mô phỏng của bạn. So sánh hai kết quả này?

In [23]:
# Lời giải

## Nhiều quầy phục vụ

Bây giờ hãy thử hai chiến lược xếp hàng khác:

1.  Một hàng với hai quầy thanh toán.
2.  Hai hàng, mỗi hàng có một quầy thanh toán.

Hình vẽ dưới đây cho thấy cả 3 kịch bản:

![](https://github.com/AllenDowney/ModSim/raw/main/figs/queue.png)

Hãy viết một hàm cập nhật cho trường hợp một hàng 2 quầy thanh toán.

In [24]:
# Lời giải

Dùng hàm cập nhật này để mô phỏng hệ thống, vẽ biểu đồ kết quả và in ra các metric.

In [26]:
# Lời giải

Vì bây giờ ta có 2 quầy thanh toán rồi, nên có thể xét đến các giá trị $\lambda$ vượt quá $\mu$.

Hãy tạo một mảng giá trị mới cho `lam` từ 10% đến 160% của `mu`.

In [27]:
# Lời giải

Hãy dùng hàm quét vừa viết để mô phỏng kịch bản 2 quầy phục vụ, một hàng đợi với một loạt các giá trị của `lam`.

Vẽ biểu đồ kết quả và in giá trị trung bình của `W` ứng với tất cả giá trị của `lam`.

In [28]:
# Lời giải

In [29]:
# Lời giải

## Nhiều hàng đợi

Để mô phỏng kịch bản với hai hàng đợi riêng, ta cần hai biến trạng thái để theo dõi số khách trong mỗi hàng.

Hãy viết một hàm cập nhật nhận vào các tham số `x1`, `x2`, `t`, `system` rồi trả lại các giá trị `x1` và `x2`. Nếu bạn không rõ cách trả lại nhiều giá trị, hãy xem `compute_metrics`.

Khi có khách đến xếp hàng, người đó sẽ chọn hàng nào?

In [30]:
# Lời giải

Hãy viết một phiên bản của `run_simulation` để làm việc với hàm cập nhật này.

In [31]:
# Lời giải

Kiểm tra hàm vừa viết bằng cách chạy mô phỏng với một giá trị của `lam`.

In [32]:
# Lời giải

Hãy quét một khoảng giá trị của `lam`, vẽ biểu đồ kết quả, rồi in ra thời gian đợi trung bình ứng với tất cả các giá trị `lam`.

Kết quả này thế nào so với kịch bản hai quầy và một hàng đợi?

In [33]:
# Lời giải

In [34]:
# Lời giải

In [27]:
# Lời giải